In [8]:
# pip install langchain langchain-experimental langchain-openai

In [9]:
AZURE_OPENAI_API_KEY = "278a70a9e90747dabc92d235cee4772d"
AZURE_OPENAI_ENDPOINT = "https://eastus.api.cognitive.microsoft.com/"

In [10]:
import os 
os.environ['AZURE_OPENAI_API_KEY'] = AZURE_OPENAI_API_KEY
os.environ['AZURE_OPENAI_ENDPOINT'] = AZURE_OPENAI_ENDPOINT


In [11]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(api_version="2024-12-01-preview",model='telcogpt-v1')

prompt = "Tell me a joke about mouse"
model.invoke(prompt)

AIMessage(content='Why did the mouse break up with his girlfriend?\n\nBecause she was a little too cheesy! 🧀🐭', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 13, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BZrCtOcdXaGzKbevWFeI5NSGXAvUB', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'

In [12]:
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}"

prompt = ChatPromptTemplate([("system",system_template),("user","{text}")])

prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [13]:
prompt.input_variables

['language', 'text']

In [14]:
prompt.invoke({"language":"Bengali","text":"HOW ARE YOU?"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Bengali', additional_kwargs={}, response_metadata={}), HumanMessage(content='HOW ARE YOU?', additional_kwargs={}, response_metadata={})])

In [16]:
# PARSERS
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
final_prompt = prompt.invoke({"language":"Bengali","text":"HOW ARE YOU?"})
op = model.invoke(final_prompt)
op

AIMessage(content='আপনি কেমন আছেন?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 20, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BZrDxHLmSv9TRoizu1C0EHhkzyULw', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severi

In [17]:
parser.invoke(op)

'আপনি কেমন আছেন?'

In [1]:
## Chain
# - a static sequence of execution involving multiple components such as LLMs, parsers, prompts, tools, loaders etc.
# - can be used to automate a rule based, linear and non linear workflows involving LLMs, tools

In [19]:
translation_chain = prompt | model | parser 

In [21]:
translation_chain.invoke({"language":"Bengali","text":"What have you eaten today?"})

'আপনি আজ কি খেয়েছেন?'

## Code Chain

In [22]:
code_generation_prompt = ChatPromptTemplate.from_template("create a python code for {text}. Return only the code and nothing else")

code_chain_01 = code_generation_prompt | model | parser

In [24]:
op = code_chain_01.invoke({"text":"fibonacci series"})
print(op)

```python
def fibonacci(n):
    fib_series = []
    a, b = 0, 1
    for _ in range(n):
        fib_series.append(a)
        a, b = b, a + b
    return fib_series

# Example usage:
n = 10
print(fibonacci(n))
```


In [25]:
code_analyze_prompt = ChatPromptTemplate.from_template("analyze the provided code and calculate the time complexity {code}")
code_chain_02 = code_analyze_prompt | model | parser

In [26]:
def patch_response (code):
    print(code)
    print("#" * 20)
    return {"code":code}     


In [27]:
final_chain = code_chain_01 | patch_response | code_chain_02

In [28]:
final_chain.invoke({'text': "fibonacci series"})

```python
def fibonacci_series(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    
    series = [0, 1]
    for i in range(2, n):
        series.append(series[-1] + series[-2])
    return series

# Example usage
n = 10
print(fibonacci_series(n))
```
####################


"Let's analyze the provided Python code for generating a Fibonacci series and determine its time complexity.\n\n### Code Walkthrough\n\nThe function `fibonacci_series(n)` generates the Fibonacci series up to the `n`-th element. \n\n1. **Base Cases**:\n   - If \\( n \\leq 0 \\), it returns an empty list.\n   - If \\( n = 1 \\), it returns a list containing only `0`.\n   - If \\( n = 2 \\), it returns a list containing `[0, 1]`.\n\n2. **General Case**:\n   - It initializes a list called `series` with the first two Fibonacci numbers, `[0, 1]`.\n   - It then enters a loop that starts from index 2 and runs until `n-1` (this means it runs \\( n-2 \\) times).\n   - For each iteration, it appends the sum of the last two elements of the list to the end of the list.\n\n### Time Complexity\n\nTo derive the time complexity, we consider the two main components:\n- The `for` loop iterates from index 2 to \\( n-1 \\), which runs \\( n-2 \\) times.\n- Each iteration of the loop performs a constant tim

## LangChain Tools

In [29]:
from langchain_community.tools  import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api = WikipediaAPIWrapper(top_k_results=3, lang="en", doc_content_chars_max= 4000)
tool = WikipediaQueryRun(api_wrapper=wiki_api)

tool.invoke("Shahrukh Khan")

'Page: Shah Rukh Khan\nSummary: Shah Rukh Khan (pronounced [ˈʃaːɦɾʊx xäːn] ; born 2 November 1965), also known by the initialism SRK, is an Indian actor and film producer who works in Hindi cinema. Referred to in the media as the "Baadshah of Bollywood" and "King Khan", he has appeared in more than 100 films, and earned numerous accolades, including 14 Filmfare Awards. He has been awarded the Padma Shri by the Government of India, as well as the Order of Arts and Letters and Legion of Honour by the Government of France. Khan has a significant following in Asia and the Indian diaspora worldwide. In terms of audience size and income, several media outlets have described him as one of the most successful film stars in the world. Many of his films thematise Indian national identity and connections with diaspora communities, or gender, racial, social and religious differences and grievances.\nKhan began his career with appearances in several television series in the late 1980s and made his 

In [30]:
print(tool.description)
print(tool.name)
print(tool.args)

A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
wikipedia
{'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}


In [ ]:
search_prompt_template = ChatPromptTemplate.from_template("generate a query to be searched on wikipedia for the user question {question}")


In [37]:
def format_wiki(topic):
    print(topic)
    return topic
wiki_chain = search_prompt_template | model | parser | format_wiki | tool | parser 

In [39]:
wiki_chain.invoke({"question" : "I want to know more about Kareena kapoor"})

You can use the following query to search on Wikipedia for more information about Kareena Kapoor:

```
Kareena Kapoor site:wikipedia.org
```

This will direct you to her Wikipedia page, where you can find detailed information about her life, career, and projects.


'Page: List of films considered the worst\nSummary: The films listed below have been cited by a variety of notable critics in varying media sources as being among the worst films ever made. Examples of such sources include Metacritic, Roger Ebert\'s list of most-hated films, The Golden Turkey Awards, Leonard Maltin\'s Movie Guide, Rotten Tomatoes, pop culture writer Nathan Rabin\'s My World of Flops, the Stinkers Bad Movie Awards, the cult TV series Mystery Science Theater 3000 (alongside spinoffs Cinematic Titanic, The Film Crew and RiffTrax), and the Golden Raspberry Awards (aka the "Razzies"). Films on these lists are generally feature-length films that are commercial/artistic in nature (intended to turn a profit, express personal statements or both), professionally or independently produced (as opposed to amateur productions, such as home movies), and released in theaters, then on home video.'